# baostock

获取K线数据

bs.query_history_k_data_plus(
    code,
    fields,
    start_date=None,
    end_date=None,
    frequency='d',
    adjustflag='3',
)

参数说明：

frequency：数据类型，默认为d，日k线；d=日k线、w=周、m=月、5=5分钟、15=15分钟、30=30分钟、60=60分钟k线数据，不区分大小写
> 指数没有分钟线数据(需要tushare补充)

> 周线每周最后一个交易日才可以获取，月线每月最后一个交易日才可以获取

In [34]:
import baostock as bs
import pandas as pd
import numpy as np

In [86]:
# 封装函数
def baostock_get_history_k_data(code,start_date,end_date,frequency,adjustflag="3"):
    lg = bs.login()
    rs = bs.query_history_k_data_plus(code,"date,open,high,low,close,preclose,volume,amount,turn,tradestatus,pctChg,isST",
                                        start_date, end_date, frequency, adjustflag)
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields, dtype=np.float)
    result["turn"] = result["turn"]*100
    result.set_index("date",inplace=True)
    del rs.fields[0]
    decimals = pd.Series([2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0], index=rs.fields)
    result = result.round(decimals)
    return result

In [76]:
lg = bs.login()

login success!


In [84]:
rs = bs.query_history_k_data_plus("sh.600000",
    "date,open,high,low,close,preclose,volume,amount,turn,tradestatus,pctChg,isST",
    start_date='2017-07-01', end_date='2017-12-31',
    frequency="d", adjustflag="3")

data_list = []
while (rs.error_code == '0') & rs.next():
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields, dtype=np.float)

result["turn"] = result["turn"]*100
result.set_index("date",inplace=True)

del rs.fields[0]
decimals = pd.Series([2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0], index=rs.fields)
result = result.round(decimals)

In [85]:
result

,open,high,low,close,preclose,volume,amount,turn,tradestatus,pctChg,isST
date,,,,,,,,,,,
2017-07-03,12.64,12.65,12.47,12.56,12.65,38778949.0,486264672.0,13.80,1.0,-0.71,0.0
2017-07-04,12.55,12.58,12.41,12.55,12.56,36659128.0,458434432.0,13.04,1.0,-0.08,0.0
2017-07-05,12.50,12.65,12.47,12.62,12.55,26470507.0,332542464.0,9.42,1.0,0.56,0.0
2017-07-06,12.62,12.72,12.51,12.66,12.62,37414241.0,471582096.0,13.31,1.0,0.32,0.0
2017-07-07,12.62,12.69,12.55,12.60,12.66,24667294.0,311101536.0,8.78,1.0,-0.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,12.61,12.68,12.56,12.59,12.62,19306662.0,243553698.0,6.87,1.0,-0.24,0.0
2017-12-26,12.57,12.66,12.56,12.64,12.59,15225722.0,192219131.0,5.42,1.0,0.40,0.0
2017-12-27,12.65,12.66,12.53,12.62,12.64,32731881.0,412197987.0,11.65,1.0,-0.16,0.0


In [10]:
rs2 = bs.query_history_k_data_plus("sh.000001",
    "date,code,open,high,low,close,preclose,volume,amount,pctChg",
    start_date='2017-01-01', end_date='2017-06-30', frequency="d")

# 综合指数，例如：sh.000001 上证指数，sz.399106 深证综指 等；
# 规模指数，例如：sh.000016 上证50，sh.000300 沪深300，sh.000905 中证500，sz.399001 深证成指等

In [8]:
sh_data = []
while (rs2.error_code == '0') & rs2.next():
    # 获取一条记录，将记录合并在一起
    sh_data.append(rs2.get_row_data())
result2 = pd.DataFrame(sh_data, columns=rs2.fields)

In [9]:
result2

,date,code,open,high,low,close,preclose,volume,amount,pctChg
0,2017-01-03,sh.000001,3105.3080,3136.4550,3105.3080,3135.9200,3103.6370,14156718592,159887138816.0000,1.040200
1,2017-01-04,sh.000001,3133.7870,3160.1020,3130.1140,3158.7940,3135.9200,16786085120,195914293248.0000,0.729400
2,2017-01-05,sh.000001,3157.9060,3168.5020,3154.2810,3165.4100,3158.7940,17472764416,199692025856.0000,0.209400
3,2017-01-06,sh.000001,3163.7760,3172.0340,3153.0250,3154.3210,3165.4100,18370896640,207296036864.0000,-0.350300
4,2017-01-09,sh.000001,3148.5310,3173.1360,3147.7350,3171.2360,3154.3210,17171407616,192110579712.0000,0.536200
...,...,...,...,...,...,...,...,...,...,...
114,2017-06-26,sh.000001,3157.0020,3187.8890,3156.9760,3185.4430,3157.8730,17357921024,200419303424.0000,0.873100
115,2017-06-27,sh.000001,3183.4190,3193.4610,3172.4630,3191.1960,3185.4430,14820119808,167523799040.0000,0.180600
116,2017-06-28,sh.000001,3183.6330,3193.4390,3170.7850,3173.2010,3191.1960,14651672064,162081099776.0000,-0.563900
117,2017-06-29,sh.000001,3174.9810,3188.7740,3174.2830,3188.0620,3173.2010,12875547136,147249799168.0000,0.468300
